In [1]:
import os
import json
from pprint import pprint
import subprocess
import shlex
import psycopg2

In [2]:
#returns the sentiment (pos, neg) when given a string.
def RateSentiment(sentiString):
    #open a subprocess using shlex to get the command line string into the correct args list format
    #p = subprocess.Popen(shlex.split("java -jar SentiStrengthCom.jar stdin sentidata C:/Python27/sentistrength/"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    p = subprocess.Popen(shlex.split("java -jar C:/Python27/SentiStrengthCom.jar stdin sentidata C:/Python27/sentistrength/"),stdin=subprocess.PIPE,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    #communicate via stdin the string to be rated. Note that all spaces are replaced with +
    stdout_text, stderr_text = p.communicate(sentiString.replace(" ","+"))
    #remove the tab spacing between the positive and negative ratings. e.g. 1    -5 -> 1-5
    stdout_text = stdout_text.rstrip().replace("\t","")
    return stdout_text

In [10]:
# Updates a JSON file
def updateJSON(newdata, outFilePath):
    data = json.load(open(outFilePath))
    if(len(data["fragments"])==0):
        with open(outFilePath, 'w') as outfile:
            json.dump(newdata, outfile, sort_keys=True, indent=4)
            print "\n\n\ndata file created: " + outFilePath
    else:
        for v in newdata["fragments"]:
            data["fragments"].append(v)
        with open(outFilePath, 'w') as outfile:
            json.dump(data, outfile, sort_keys=True, indent=4)
            print "\n\n\n==================data file updated: " + outFilePath

In [26]:
# Gets the sentences in JSONs, applies sentiment analysis and updates the JSON objects.
def getSentencesFromJSON(jsonFilePath, company):
    data = json.load(open(jsonFilePath))
    pprint(data["fragments"][0])
    QnAPartCounter = 0
    QuestionCounter = 0
    AnswerCounter = 0
    for v in data["fragments"]:
        #print v['lines'][0]
        sentence = v['lines'][0]
        if len(sentence)>0:
            pos, neg = RateSentiment(sentence).split('-')
        else:
            pos = '0'
            neg = '0'
        print sentence + "," + pos + "," + neg
        v['positive_sentiment'] = pos
        v['negative_sentiment'] = neg
        v['sentenceMP3'] = jsonFilePath
        v['company'] = company
        yearquart=jsonFilePath
        if 'qna' in yearquart:
            year = yearquart[yearquart.find('qna')-3:yearquart.find('qna')]
        else:
            year = yearquart[yearquart.find('call')-3:yearquart.find('call')]
        v['year'] = '20' + year[1:]
        v['quartile'] = year[0]
        v['yearQuartile'] = '20' + year[1:] + year[0]
        
        if "O: " in sentence:
            QnAPartCounter += 1
            QuestionCounter = 0
            AnswerCounter = 0
        if "Q: " in sentence:
            QuestionCounter += 1
            AnswerCounter = 0
        if "A: " in sentence:
            AnswerCounter += 1
        v["question_id"] = company + "." + '20' + year[1:] + year[0] + "." + str(QnAPartCounter) + "." + str(QuestionCounter) + "." + str(AnswerCounter)
    updateJSON(data, "data_sentiment_deneme.json")

In [28]:
# Walkthrough the files in the folder and subfolders
def pywalkerMP3(path, company):
    for root, dirs, files in os.walk(path):
        for file_ in files:
            if("qna.json" in file_):
                filename = os.path.join(root, file_) 
                print filename
                getSentencesFromJSON(filename, company)

# ENTRANCE METHOD. Give the Folder that contains JSON files.
pywalkerMP3('G:\\SES\\sesq416qna\\', 'SES')

G:\SES\sesq416qna\sesq416qna.json
{u'begin': u'0.000',
 u'children': [],
 u'end': u'4.920',
 u'id': u'f000001',
 u'language': u'eng',
 u'lines': [u'Q: Yeah.']}
Q: Yeah.,1,1
So, Wilton, there's no correlation between what Charlie Ergen says and maturity on our bonds, so it's the first point I'd like to make.,2,1
And just to remind you of our financial framework and our treasury road map, we've a very consistent treasury road map.,2,1
We work off of three or four principles which will continue to prevail.,1,1
One is we're a long-term business and, therefore, we've worked over the last two to three years to increase the overall maturity of our senior debt such that it's now around eight years.,1,1
We did that but at the same time reducing the average cost of our debt towards the 3.87% today.,1,1
Secondly, we keep a good mix between dollar-denominated debt and euro-denominated debt so that we match this with our dollar-denominated EBITDA and euro-denominated EBITDA.,3,1
And you'll see, whe

In [36]:
import json
from pprint import pprint

#nq111qna_f000001.mp3
def addEmotionVecToJSON(jsonFilePath, emotionFilePath):
    data = json.load(open(jsonFilePath))
    dataEmo = json.load(open(emotionFilePath))
    print "No of sentences: " + str(len(data["fragments"]))
    print "No of emotion readings: " + str(len(dataEmo))
    mp3List = []
    i = 0
    for v in dataEmo:
        if v["filename"] not in mp3List:
            longfilename = v["filename"]
            fname =longfilename[longfilename.find('mp3\\')+4:]
            for x in data["fragments"]:
                fileextract = x["sentenceMP3"]
                firstpartfile = fileextract[fileextract.find('qna\\')+4:]
                idpart = x["id"]
                #print 'this? ' + firstpartfile
                filenamemp3 = firstpartfile[:-5] + "_" + idpart + ".mp3"
                #print 'filename: ' + fname
                #print 'filenamemp3: ' + filenamemp3
                if fname == filenamemp3:
                    i=i+1 
                    x["elisaEmoVec"] = v
                    #print str(i) + ") " + filenamemp3 + " " + str(v)
                    mp3List.append(fname)
    print len(mp3List)
    #print mp3List[1]
    
    outFilePath = 'ses_data_with_emovec.json'
    with open(outFilePath, 'w') as outfile:
        json.dump(data, outfile, sort_keys=True, indent=4)
        print "\n\n\n==================data file updated: " + outFilePath

    return 0

addEmotionVecToJSON('SES.json', 'jsn_ses.json')

No of sentences: 5595
No of emotion readings: 5582
5582



==================data file updated: ses_data_with_emovec.json


0

In [41]:
import csv
import json

def returnDateFromQuartile(dictionary, quartile, company):
    return dict[company.lower() + "_" + quartile]

dict = {}
with open('price_and_position_data\\datequartmap.csv', 'rb') as csvfile:
    quartmap = csv.reader(csvfile, delimiter=';', quotechar='"')
    for row in quartmap:
        dict[row[0].lower() + "_" + row[1] ]=row[2]

#print returnDateFromQuartile(dict, '20121', 'BT')

jsonFilePath = 'demo\\ses_master.json'
outFilePath = 'demo\\ses_master_calldate.json'

data = json.load(open(jsonFilePath))
print "No of sentences: " + str(len(data["fragments"]))
i = 0
for x in data["fragments"]:
    i = i + 1
    yearquart = x["yearQuartile"]
    comp = x["company"]
    x["callDate"] = returnDateFromQuartile(dict, yearquart, comp) 

with open(outFilePath, 'w') as outfile:
    json.dump(data, outfile, sort_keys=True, indent=4)
    print "\n\n\n==================data file updated: " + outFilePath   

No of sentences: 5595



==================data file updated: demo\ses_master_calldate.json


In [56]:
question_id = "Nokia.20111.1.0.0"

def sentenceTypeFits(sentenceType, question_id):
	returnVal = False
	answer_bit = question_id[question_id.rfind(".")+1:]
	question_bit = question_id[question_id.rfind(".", 0, question_id.rfind("."))+1:question_id.rfind(".")]
	print question_bit
	print answer_bit
	#Q
	if 'Q' in sentenceType and 'A' not in sentenceType and 'O' not in sentenceType: 
		if question_bit != '0' and answer_bit == '0':
			returnVal = True
	#A
	if 'Q' not in sentenceType and 'A' in sentenceType and 'O' not in sentenceType: 
		if answer_bit != '0':
			returnVal = True
	#O
	if 'Q' not in sentenceType and 'A' not in sentenceType and 'O' in sentenceType: 
		if question_bit == '0' and answer_bit == '0':
			returnVal = True
	#QA
	if 'Q' in sentenceType and 'A' in sentenceType and 'O' not in sentenceType: 
		if question_bit != '0' or answer_bit != '0':
			returnVal = True
	#QO
	if 'Q' in sentenceType and 'A' not in sentenceType and 'O' in sentenceType: 
		if answer_bit == '0':
			returnVal = True
	#AO
	if 'Q' not in sentenceType and 'A' in sentenceType and 'O' in sentenceType: 
		if question_bit == '0':
			returnVal = True
	#QAO
	if 'Q' in sentenceType and 'A' in sentenceType and 'O' in sentenceType: 
		returnVal = True

	return returnVal

print sentenceTypeFits('QA', question_id)

0
0
False
